<a href="https://colab.research.google.com/github/yukiharada1228/neural_network/blob/main/StableLM%E3%82%92%E8%A9%A6%E3%81%97%E3%81%AB%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPUの確認
import torch

is_cuda = torch.cuda.is_available()
device_name = ""
if is_cuda:
    device_name = torch.cuda.get_device_name()
print({"is_cuda": is_cuda, "device_name": device_name})

{'is_cuda': True, 'device_name': 'Tesla T4'}


In [2]:
# 必要なライブラリをインストール
%%capture
%pip install -q sentencepiece einops transformers huggingface_hub accelerate bitsandbytes

In [3]:
# StableLMを使用するため許可されたアカウントでhuggingfaceにログイン
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
from transformers import LlamaTokenizer, AutoModelForCausalLM

# チャットボットを作成
class StableChatBot:
    def __init__(self, max_length=128, k=40):
        tokenizer_id = "novelai/nerdstash-tokenizer-v1"
        model_id = "stabilityai/japanese-stablelm-instruct-alpha-7b"
        self.tokenizer = LlamaTokenizer.from_pretrained(tokenizer_id, additional_special_tokens=['▁▁'])
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            device_map="auto",
            variant="int8",
            load_in_8bit=True
        )
        self.max_length = max_length
        self.k = k

    def chat(self):
        self.model.eval()
        while "[exit]" not in (user_message := self._input()):

            prompt = self.make_prompt(user_message)

            token_ids = self.tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

            output = ""
            for word in self.gradually_generate(token_ids):
                print(word, end='', flush=True)
                output += word
            print()

    def _input(self):
        s = input("> ").replace("\r", "").replace("\n", "")
        return s

    def make_prompt(self, user_message, sep="\n\n### "):
        sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
        p = sys_msg
        p += sep + "指示"
        p += '\n' + user_message
        p += sep + "応答"
        return p

    def gradually_generate(self, token_ids):
        token_ids = token_ids.to(self.model.device)
        for _ in range(self.max_length):
            with torch.no_grad():
                outputs = self.model(token_ids)

            logits = outputs.logits
            indices_to_remove = logits < torch.topk(logits, self.k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
            probs = torch.nn.functional.softmax(logits[..., -1, :], dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            token_ids = torch.cat((token_ids, next_token_id), dim=-1)

            output_str = self.tokenizer.decode(next_token_id[0])

            yield output_str

            if "<|endoftext|>" in output_str:
                break

In [5]:
# チャットボットを作成
bot = StableChatBot()

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b:
- configuration_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b:
- modeling_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [6]:
# チャットを開始
bot.chat()

> CNNとはどんなアーキテクチャですか

CNNアーキテクチャは、ニューラルネットの一種です。CNNは画像データセットに基づいて構築され、画像を認識して分類するときに、データを圧縮して小さなデータに置き換えるための特別な処理を伴います。CNNの主な利点の1つは、モデルがデータの学習とテストに優れているため、認識と認識が高速であることです。<|endoftext|>
> 画像処理について教えてください

画像の画像処理（画像を修正するために画像に加えられた変更）は、画像の明るさ、コントラスト、および色を変えることを含んでいます。<|endoftext|>
> [exit]
